# Gaussian Updated Fields + Analytics

In [1]:
import sys
import numpy as np 
sys.path.append('..')

In [2]:
# Loading my modules
from plotting_tools.plotter import plotter, cross_section_plotter
from plotting_tools.field_comparison import differencer
from physics_informed_vector_fields.current_carrying_wire import field_around_current_carrying_wire
from physics_informed_vector_fields.current_carrying_coil import field_around_current_carrying_coil
from GP_Implementation.Kernel import updated_vector_field

In [3]:
# getting the fields to draw samples from
outputs_wire = field_around_current_carrying_wire(100000000)
outputs_coil = field_around_current_carrying_coil(100000000)

## Determining Sampling Points

In [4]:
def sample_field(inputs, n_samples: int, random_seed: int = 0):
    '''
    Helper function to use to get random points from earlier fields. Samples n points from the given field data. 

    Args: 
        inputs[0], inputs[1], inputs[2] (np.ndarray): x, y, z Meshgrid coordinates.
        inputs[3], inputs[4], inputs[5] (np.ndarray): Bx, By, Bz Magnetic field components.
        n_samples (int): Number of points to sample.

    Returns:
        list of nd.arrays of sampled x, y and zs and fields at these points
    '''
    x, y, z, Bx, By, Bz = inputs[0], inputs[1], inputs[2], inputs[3], inputs[4], inputs[5]
    np.random.seed(random_seed)
    
    resolution = x.shape[0]
    indices = np.random.choice(resolution * resolution * resolution, size=n_samples, replace=False)
    
    x_samples = x.flatten()[indices]
    y_samples = y.flatten()[indices]
    z_samples = z.flatten()[indices]
    Bx_samples = Bx.flatten()[indices]
    By_samples = By.flatten()[indices]
    Bz_samples = Bz.flatten()[indices]
    
    return x_samples, y_samples, z_samples, Bx_samples, By_samples, Bz_samples

In [5]:
sampled_outputs_wire = sample_field(outputs_wire, n_samples=100)
plotter(sampled_outputs_wire, 
        title = 'Points given to our GP Regressor for current carrying wire')

In [6]:
sampled_outputs_coil = sample_field(outputs_coil, n_samples=100)
plotter(sampled_outputs_coil, 
        title = 'Points given to our GP Regressor for current carrying loop')

## Applying GP Regression to Current Carrying Wire

In [7]:
# Divergence free attempt
outputs_wire_div_free= updated_vector_field(
    sampled_outputs_wire, 
    outputs_wire[0], outputs_wire[1], outputs_wire[2], 
    kernel_func='divergence_free_kernel', 
    sigma_f =  0.005, l = 0.8)


# from giving the code a large number of samples, the optimum 
# parameters seem to be about: sigma_f = 0.1, l = 1.2323 (this is from mle and it works poorly)
# sigma_f = 0.005, l = 0.8 seem to work okay 

Parameters for sigma_f and l specified, using these.


In [8]:
given_samples_wire = sampled_outputs_wire[0].shape[0]
plotter(outputs_wire_div_free, 
        title = f'Div free output from GP Regression for n = {given_samples_wire} input vectors for current carrying wire')

In [9]:
# plot residuals of fit above: 
residuals_wire_div_free, mse_wire_div_free = differencer(outputs_wire, outputs_wire_div_free)
print(f'MSE for this case of {mse_wire_div_free}')
plotter(residuals_wire_div_free, 
        title = f'Div free regression errors for n = {given_samples_wire} input vectors for current carrying wire')

MSE for this case of 31.21112248201693


Errors are largest inside of the coil. The fields here are at their largest and vary most rapidly, hence 

In [10]:
# these curl ones work very poorly in general
outputs_wire_curl_free = updated_vector_field(
    sampled_outputs_wire, 
    outputs_wire[0], outputs_wire[1], outputs_wire[2], 
    kernel_func='curl_free_kernel', 
    sigma_f = 0.1, l = 0.3)

Parameters for sigma_f and l specified, using these.


In [11]:
plotter(outputs_wire_curl_free, 
        title = f'Curl free output from GP Regression for n = {given_samples_wire} input vectors for current carrying wire')

In [12]:
# plot residuals of fit above: 
residuals_wire_curl_free, mse_wire_curl_free = differencer(outputs_wire, outputs_wire_curl_free)
print(f'MSE for this case of {mse_wire_curl_free}')
plotter(residuals_wire_curl_free, 
        title = f'Curl free regression errors for n = {given_samples_wire} input vectors for current carrying wire')

MSE for this case of 367.4297374089136


## Applying GP Regression to Current Carrying Coil

In [13]:
given_samples_coil = sampled_outputs_coil[0].shape[0]

outputs_coil_div_free = updated_vector_field(
    sampled_outputs_coil, 
    outputs_coil[0], outputs_coil[1], outputs_coil[2], 
    kernel_func='divergence_free_kernel', 
    sigma_f = 0.01, l = 1.5)

# from giving the code a large number of samples, the optimum 
# parameters seem to be about: sigma_f = 0.1, l = 0.1191608
# sigma_f = 0.01, l = 1.5 seem to work okay 

Parameters for sigma_f and l specified, using these.


In [14]:
plotter(outputs_coil_div_free, 
        title = f'Div free output from GP Regression for n = {given_samples_coil} input vectors for current carrying coil')

In [15]:
# plot residuals of fit above: 
residuals_coil_div_free, mse_coil_div_free = differencer(outputs_coil, outputs_coil_div_free)
print(f'MSE for this case of {mse_coil_div_free}')
plotter(residuals_coil_div_free, 
        title = f'Div free regression errors for n = {given_samples_coil} input vectors for current carrying coil')

MSE for this case of 20.614333604296107


Interestingly we can see this does well everywhere apart from nearby to the conductor. I think this is becuase the length scale which would be good here is different to other places. 

In [16]:
outputs_coil_curl_free= updated_vector_field(
    sampled_outputs_coil, 
    outputs_coil[0], outputs_coil[1], outputs_coil[2], 
    kernel_func='curl_free_kernel',
    sigma_f = 0.1, l = 0.3)

Parameters for sigma_f and l specified, using these.


In [17]:
plotter(outputs_coil_curl_free, 
        title = f'Curl free output from GP Regression for n = {given_samples_coil} input vectors for current carrying coil')

In [18]:
# plot residuals of fit above: 
residuals_coil_curl_free, mse_coil_curl_free = differencer(outputs_coil, outputs_coil_curl_free)
print(f'MSE for this case of {mse_coil_curl_free}')
plotter(residuals_coil_curl_free, 
        title = f'Curl free regression errors for n = {given_samples_coil} input vectors for current carrying coil')

MSE for this case of 143.3251164482789
